In [5]:
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
import sys
sys.path.append('../library/')
from models import *
from Epi_models import*
from functions import *
import networkx as nx
import matplotlib.animation as animation
import seaborn
import pickle
import warnings
warnings.filterwarnings('ignore')
import matplotlib.ticker as ticker
Text_files_path = '../Text_files/Testing_Networks/'
%autosave 60

Autosaving every 60 seconds


# Stochastic Model

In [6]:
N = 2000
graphs_names = np.array(['barabasi-albert','watts-strogatz'])
G1 = nx.barabasi_albert_graph(N, 2)
G2 = nx.watts_strogatz_graph(N, 4, 1)

#----Load data network of contacts----
N = 2000
graphs_names = np.array(['barabasi-albert','watts-strogatz'])
infile_k = open(Text_files_path+'Stochastic/Networks/barabasi-albert/k.pck','rb')
k = pickle.load(infile_k)
infile_k.close()
infile_p_k = open(Text_files_path+'Stochastic/Networks/barabasi-albert/p_k.pck','rb')
p_k = pickle.load(infile_p_k)
infile_p_k.close()
meanDegree = np.sum(k*p_k)
meanDegree2 = np.sum(k**2*p_k)
print(meanDegree, meanDegree2, np.max(k))

3.9960271618768215 47.23170201521739 265


## Ensemble

In [8]:
sample_sizes = [int((N/100)*1.875), int((N/100)*3.125), int((N/100)*5.0)]
print('Sample Sizes:',sample_sizes)
colors = ['b', 'g', 'r']
colors2 = ['purple', 'violet', 'brown']
N = 2000
print('N:', N)
gamma = 1/6
print('Gamma:', gamma)
sigma = 1/4
print('Sigma:', sigma)
graphs = [G1]
graphs_names = np.array(['barabasi-albert','watts-strogatz'])
sampling_protocol = np.array(['uniform', 'aposteriori'])
folders  = np.array(['../Figures/Control/Stochastic/Ensemble/Networks/', '../Figures/Stochastic/Networks/'])

#ps = np.array([0.0, 1.0])
ps = np.array([0.0, 1.0])
print('ps:',ps)
R0s = np.array([1.2, 2.0, 3.0, 4.5])
R0s2 = np.array([0.8, 1.2, 2.0, 3.0, 4.5]) # with network p=0

n_sims = np.array([8000, 8000, 8000, 8000, 8000])
#n_sims = np.array([8000, 8000, 8000, 6000, 6000])
print('n_sim:',n_sims)

Betas = np.array([R0s2*gamma,R0s*gamma])
print('Betas:', Betas)

R0_Ns = (R0s2*(meanDegree2-meanDegree))/(meanDegree*(meanDegree+R0s2))
R0_Es = np.sqrt(1-4*((sigma*gamma-sigma*Betas[1])/(sigma + gamma)**2))
R0_Es2 = np.sqrt(1-4*((sigma*gamma-sigma*Betas[0])/(sigma + gamma)**2))
R0_ENs = (R0_Es2*(meanDegree2-meanDegree))/(meanDegree*(meanDegree+R0_Es2))

print('R0s:', R0s)
print('R0_Es:',R0_Es)
print('R0_Ns:',R0_Ns)
print('R0_ENs:',R0_ENs)

lambdas = ((-sigma-gamma)/(2)) + (1/2)*np.sqrt((sigma-gamma)**2 + 4*sigma*Betas[1]) #exponential growth rates

ests = (1/(lambdas)).astype(int)
if(sigma==1/4):
    est_Ns = (1/((R0_ENs-1)*gamma)).astype(int)
    R0ss = R0_ENs
if(sigma==1000):
    est_Ns = (1/((R0_Ns-1)*gamma)).astype(int)
    R0ss = R0_Ns

Ts_Total = np.array([5*est_Ns,5*ests]) #for R0 = [1.2, 1.5, 3.0, 4.5] use [160, 35, 20] and half of it if when sigma=1000
print('Times:',Ts_Total)

#----Critical occupancy----
T_c = meanDegree/(meanDegree2-meanDegree)
u = np.linspace(0.00005,0.9,100000)

aposterioris = np.array([False, True])
protocols = np.array([0, 1])
print(aposterioris, protocols)

Sample Sizes: [37, 62, 100]
N: 2000
Gamma: 0.16666666666666666
Sigma: 0.25
ps: [0. 1.]
n_sim: [8000 8000 8000 8000 8000]
Betas: [array([0.13333333, 0.2       , 0.33333333, 0.5       , 0.75      ])
 array([0.2       , 0.33333333, 0.5       , 0.75      ])]
R0s: [1.2 2.  3.  4.5]
R0_Es: [1.09178753 1.4        1.70880075 2.0880613 ]
R0_Ns: [1.80477125 2.49875481 3.60894459 4.63963246 5.7307364 ]
R0_ENs: [1.9868922  2.3217778  2.80716357 3.24087804 3.7133128 ]
Times: [array([30, 20, 15, 10, 10]) array([260,  60,  30,  20])]
[False  True] [0 1]


In [13]:
%%time
for aposteriori, protocol in zip(aposterioris, protocols):
    print('Aposteriori:', aposteriori, ' Protocol: ', sampling_protocol[protocol])
    ##Run over values of p
    for p, Ts_total, betas in zip (ps, Ts_Total, Betas):
        ##Run over graphs
        for G, G_name in zip(graphs, graphs_names):
            ##Run over R0s
            for beta, T_total, n_sim in zip (betas, Ts_total, n_sims):
                print('p:',p)
                print('R0*:', np.sqrt(1-4*((sigma*(gamma-beta))/(sigma+gamma)**2)))
                slope = np.ones_like(k)
                if(aposteriori):
                    #T = 1-np.sum(p_k*(1/(1+(np.sqrt(1-4*((sigma*gamma-sigma*beta)/(sigma+gamma)**2)))/(k**2))))
                    if(sigma == 1000):
                        tau = 1/gamma
                    if(sigma == 1/4):
                        tau = 2*(sigma+gamma)**(-1)
                    T = 1-np.sum(p_k*(1/(1+((beta*tau)/(k*(k))))))
                    u_sol = u[np.array([np.sum(p_k*k*(1+(i-1)*T)**(k-1)) for i in u])>(np.sum(p_k*k)*u)][-1]
                    S = 1 - np.sum(p_k*(1-T+(T*u_sol))**k)**2
                    e_k = 1-(1-T+(T*u_sol))**(2*k)
                    slope = e_k/S
                    #slope = slope - slope[0]
                    #print('slope:',slope)
                print('Succ nodes:')
                ### Generate new graph
                G = nx.barabasi_albert_graph(N, 2)
                ### run function
                %time run_network_ensemble(N=N, G=G, G_name=G_name, beta=beta, sigma=sigma, gamma=gamma, T_total=T_total,intervals=T_total+1, n_ensemble=n_sim, p = p, initE=0, initI=1, folder =Text_files_path+'Sampling/Networks/'+G_name+'/'+sampling_protocol[protocol]+'/k_normalization/likelihood', stochastic = False, sampling = True, sample_sizes = sample_sizes, aposteriori= aposteriori, slope = slope)
                print('\n')

Aposteriori: False  Protocol:  uniform
p: 0.0
R0*: 0.8988882021697693
Succ nodes:
8000
CPU times: user 6min 12s, sys: 18.5 s, total: 6min 31s
Wall time: 6min 32s


p: 0.0
R0*: 1.0917875251164944
Succ nodes:
0
CPU times: user 6min 14s, sys: 17.6 s, total: 6min 31s
Wall time: 6min 32s


p: 0.0
R0*: 1.4000000000000001
Succ nodes:
18
CPU times: user 6min 17s, sys: 17.1 s, total: 6min 34s
Wall time: 6min 35s


p: 0.0
R0*: 1.7088007490635064
Succ nodes:
81
CPU times: user 6min 10s, sys: 17.1 s, total: 6min 27s
Wall time: 6min 28s


p: 0.0
R0*: 2.08806130178211
Succ nodes:
561
CPU times: user 6min 29s, sys: 17.3 s, total: 6min 46s
Wall time: 6min 47s


p: 1.0
R0*: 1.0917875251164944
Succ nodes:
0
CPU times: user 25min 17s, sys: 20.1 s, total: 25min 37s
Wall time: 25min 38s


p: 1.0
R0*: 1.4000000000000001
Succ nodes:
3778
CPU times: user 46min 22s, sys: 25.4 s, total: 46min 47s
Wall time: 46min 50s


p: 1.0
R0*: 1.7088007490635064
Succ nodes:
4796
CPU times: user 28min 2s, sys: 20.8 s, total:

IndexError: index 266 is out of bounds for axis 0 with size 265



p: 1.0
R0*: 1.0917875251164944
Succ nodes:
0
CPU times: user 43min 47s, sys: 21.1 s, total: 44min 8s
Wall time: 44min 11s


p: 1.0
R0*: 1.4000000000000001
Succ nodes:
3696
CPU times: user 58min 5s, sys: 25.3 s, total: 58min 30s
Wall time: 58min 33s


p: 1.0
R0*: 1.7088007490635064
Succ nodes:
4793
CPU times: user 35min 55s, sys: 21.9 s, total: 36min 17s
Wall time: 36min 19s


p: 1.0
R0*: 2.08806130178211
Succ nodes:
5827
CPU times: user 33min 28s, sys: 21.7 s, total: 33min 49s
Wall time: 33min 50s


CPU times: user 6h 8min 27s, sys: 5min 34s, total: 6h 14min 2s
Wall time: 6h 14min 18s
